In [70]:
%matplotlib widget

In [188]:
import torch
import pyro
from pyro.infer import mcmc

In [189]:
import numpy

In [190]:
from collections import OrderedDict

In [205]:
def half_life_one_isotope(obs):
    daily_body_waste = pyro.sample('daily_body_waste', pyro.distributions.Exponential(100.))
    half_life = pyro.sample('half_life', pyro.distributions.Exponential(1.))
    
    likelihoods = []
    l0 = obs['l0']
    t0 = obs['t0']
    for fi, fo in enumerate(obs['followup'][1:]):
        t_ = fo['time']-t0
        level_ = fo['level']
    
        mu_ = ((1./2.) ** (t_ / half_life)) * l0 - daily_body_waste * t_
        lik_ = pyro.sample('obs_{}'.format(fi), pyro.distributions.Normal(mu_, 1.), obs=torch.ones(1) * level_) 
        likelihoods.append(lik_)
    
    return likelihoods

In [206]:
def convert_gdays_hours(inp):
    out = []
    for ii in inp:
        days = numpy.floor(ii)
        hours = (ii - days)
        out.append(24 * (hours + days))
    return out

In [207]:
''' I-123 '''
timestamps = [43618.65625,
            43618.70833,
            43618.74653,
            43618.82292,
            43618.89236,
            43618.95486,
            43619.38889,
            43619.4375]
# measurements = [
#     40,
# 40,
# 43,
# 37,
# 34,
# 31.6,
# 5.3,
# 6.77
# ]
measurements = [
    6600,
    6300,
    6600,
    5800,
    5300,
    4900,
    850,
    1050,
]

# ''' i-131 '''
# timestampes = [43619.67431,
# 43619.78472,
# 43620.34792,
# 43620.46181,
# 43620.52083,
# 43620.73958,
# 43621.38542,
# 43621.69861,
# 43622.64583]
# measurements = [1640,
# 1560,
# 750,
# 740,
# 650,
# 477,
# 208,
# 178,
# 117,
# ]
# measurements = [840000,
# 575000,
# 370000,
# 380000,
# 304000,
# 208000,
# 92000,
# 66200]

In [208]:
timestamps = convert_gdays_hours(timestamps)

In [209]:
obs = OrderedDict()

In [210]:
obs['t0'] = timestamps[0]
obs['l0'] = measurements[0]

In [211]:
obs['followup'] = []
for tt, mm in zip(timestamps, measurements):
    obs['followup'].append(OrderedDict({'time': tt, 'level': mm}))

In [212]:
nuts_kernel = pyro.infer.mcmc.NUTS(half_life_one_isotope, adapt_step_size=True, step_size=0.1)
mcmc_run = pyro.infer.mcmc.MCMC(nuts_kernel, num_samples=200, warmup_steps=50).run(obs)
score_marginal = pyro.infer.EmpiricalMarginal(mcmc_run, sites=["half_life", "daily_body_waste"])

Sample: 100%|██████████| 250/250 [14:17<00:00,  3.73s/it, step size=7.76e-05, acc. rate=0.976]


In [213]:
print('estimated half life (hours)', score_marginal.mean[0].item(), '±', 24 * score_marginal.stddev[0].item())

estimated half life (hours) 10.359722137451172 ± 0.04223825968801975


In [214]:
print('estimated cpm body exit per day (not accounted by radioactive decay)', score_marginal.mean[1].item(), '±', score_marginal.stddev[1].item())

estimated cpm body exit per day (not accounted by radioactive decay) 0.0042134542018175125 ± 5.64829315408133e-05


In [204]:
score_marginal.mean

tensor([2.5858e+04, 3.2370e-03])